<a href="https://colab.research.google.com/github/malborroni/RECMojion/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerie

In [3]:
! sudo pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-t6vcr5z7
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-t6vcr5z7
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=be267561bd8e2a32dab5b222d6ae93e1b8b9b6bd1739bbc36ddd37a6272b9314
  Stored in directory: /tmp/pip-ephem-wheel-cache-_p_yz70t/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
# Example of face detection with a vggface2 model
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import random as rnd

import seaborn as sns
from matplotlib import style
style.use('seaborn-pastel')

import keras
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input, decode_predictions
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold

import random
import pickle

In [0]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

# Funzioni

In [0]:
def graph(history):
  # funzione che prende in input il modello fittato e fa un plot della loss in validation e train
  epochs=len(history.history['loss'])
  x_plot = list(range(1,epochs+1))
  plt.figure()
  
  plt.subplot(1, 2, 1)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(x_plot, history.history['loss'])
  plt.plot(x_plot, history.history['val_loss'])
  plt.legend(['Training', 'Validation'])

  
  plt.subplot(1, 2, 2)
  plt.xlabel('Epochs')
  plt.ylabel('Acc')
  plt.plot(x_plot, history.history['acc'])
  plt.plot(x_plot, history.history['val_acc'])
  plt.legend(['Training', 'Validation'])
  
  plt.tight_layout()
  plt.show()


#Dataset

Il dataset si trova in un file chiamato XY.pkl nel drive 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = np.load('/content/drive/My Drive/data_prep.npz')

In [9]:
# training set
x_train = data['x_train_fe']
y_train = data['y_train']

# validation set
x_val = data['x_val_fe']
y_val = data['y_val']

# test set
x_test = data['x_test_fe']
y_test = data['y_test']

print('Shape del training set: {} --- {}\nShape del validation set: {} --- {}\nShape del test set: {} --- {}'. format(x_train.shape, y_train.shape,
                                                                                                                 x_val.shape, y_val.shape,
                                                                                                                 x_test.shape, y_test.shape))

Shape del training set: (2157, 14, 14, 1024) --- (2157,)
Shape del validation set: (415, 14, 14, 1024) --- (415,)
Shape del test set: (518, 14, 14, 1024) --- (518,)


In [0]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = to_categorical(y_train, num_classes = 7)
y_test = le.fit_transform(y_test)
y_test = to_categorical(y_test, num_classes = 7)
y_val = le.fit_transform(y_val)
y_val = to_categorical(y_val, num_classes = 7)

In [0]:
def create_model(C=1,D=1):
  # Create model
  model = Sequential()
  # C1
  model.add(Conv2D(32, kernel_size=1, activation='relu', input_shape=(14, 14, 1024)))
  model.add(Conv2D(128, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, kernel_size=1, activation='relu'))
  # C2
  if(C>1):
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=1, activation='relu'))
  # C3
  if(C>2):
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(Conv2D(32, kernel_size=1, activation='relu'))
  
  model.add(Flatten())
  
  # D1
  model.add(Dense(112, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dropout(0.5))
  # D2
  if(D>1):
    model.add(Dense(56, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
  # D3
  if(D>2):
    model.add(Dense(21, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
  
  model.add(Dense(7, activation='softmax'))

  # Compile model
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['acc'])
  
  # name
  if C == 2 and D == 3:
    string_name = 'C1-C2-D1-D2-D3'
    model.name = string_name
  if C == 1 and D == 3:
    string_name = 'C1-D1-D2-D3'
    model.name = string_name
  if C == 2 and D == 2:
    string_name = 'C1-C2-D1-D2'
    model.name = string_name
  if C == 1 and D == 2:
    string_name = 'C1-D1-D2'
    model.name = string_name

  return model

In [0]:
results = dict()

In [46]:
# config model
batch_size = 128
num_epoch = 100


acc_scores_tmp= []
loss_scores_tmp = []
fold = 10

for i in range(fold):
  model = create_model(C=1, D=2)
  name_model = model.name
  print('Start {}/{} in Model: {}'.format(i+1, fold, name_model))
  # callbacks
  filepath = "/content/drive/My Drive/Colab Notebooks/AML/{}.h5".format(name_model)
  checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
  early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1,restore_best_weights=True) 
  callbacks_list = [early_stop, checkpoint]
  
  model.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adam(),
                    metrics=['acc'])
        
  # Fit the model
  model_log = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=0,
          validation_data=(x_val, y_val),
          callbacks=callbacks_list)

  # visualize graph
  # graph(model_log)

  # Evaluate the model
  # model.load_weights(filepath)
  scores = model.evaluate(x_test, y_test, verbose=1)
  accuracy = 100*scores[1]
  loss = scores[0]
  print("Post-training accuracy: {} %".format(round(accuracy, 3)))
  print("Post-training loss: {}".format(round(loss, 4)))
  acc_scores_tmp.append(accuracy)
  loss_scores_tmp.append(loss)

results[model.name] = (np.mean(acc_scores_tmp), np.std(acc_scores_tmp), np.mean(loss_scores_tmp), np.std(loss_scores_tmp))


Start 1/10 in Model: C1-D1-D2
Restoring model weights from the end of the best epoch
Epoch 00026: early stopping
518/518 [==============================] - 0s 484us/step
Post-training accuracy: 91.313 %
Post-training loss: 0.507
Start 2/10 in Model: C1-D1-D2
Restoring model weights from the end of the best epoch
Epoch 00023: early stopping
518/518 [==============================] - 0s 487us/step
Post-training accuracy: 90.927 %
Post-training loss: 0.5191
Start 3/10 in Model: C1-D1-D2
Restoring model weights from the end of the best epoch
Epoch 00021: early stopping
518/518 [==============================] - 0s 513us/step
Post-training accuracy: 89.768 %
Post-training loss: 0.6003
Start 4/10 in Model: C1-D1-D2
Restoring model weights from the end of the best epoch
Epoch 00033: early stopping
518/518 [==============================] - 0s 498us/step
Post-training accuracy: 91.506 %
Post-training loss: 0.4507
Start 5/10 in Model: C1-D1-D2
Restoring model weights from the end of the best ep

In [47]:
results

{'C1-C2-D1-D2': (89.7490347582401,
  0.8256140956987499,
  0.580214705200269,
  0.06162796557299608),
 'C1-C2-D1-D2-D3': (86.31274132539866,
  4.834317586350704,
  0.6827254400059983,
  0.10628690698927971),
 'C1-D1-D2': (91.04247105167642,
  0.6808954471733075,
  0.4976640829938718,
  0.06634651321456955),
 'C1-D1-D2-D3': (90.65637066557602,
  1.1130143019526728,
  0.6422132091172414,
  0.07035462879079615)}

In [0]:
results_copy = results.copy()

In [0]:
import json

In [0]:
json = json.dumps(results)

f = open("/content/drive/My Drive/Colab Notebooks/AML/result_architecture_selection_max.json","w")
f.write(json)
f.close()

In [0]:
df = pd.DataFrame(results)

In [0]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/AML/result_architecture_selection_max.csv')